In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
import random
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from math import ceil, floor
import cv2


import tensorflow as tf
import keras

import sys


def reduce_mem_usage(df):
    # iterate through all the columns of a dataframe and modify the data type
    #   to reduce memory usage.        
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


def import_data(file):
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

****The preprocessing of dicom images

In [ ]:
train_images_dir="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/"
test_images_dir="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_test/"


The helper functions, which are used for the image data augmentations.

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa    

Read the traning data, stage_1_train_images.

In [ ]:
def read_training_data(train_dir):
    train_df=import_data(train_dir)
    train_df["SubType"]=train_df["ID"].apply(lambda x: x.rsplit("_",1)[-1])
    train_df["ID"]=train_df["ID"].apply(lambda x: x.rsplit("_",1)[0])
    train_df.drop_duplicates(inplace=True)
    Label=np.reshape(train_df["Label"].values,(-1,6))
    train_df=pd.DataFrame(Label,
                          columns=list(train_df["SubType"].unique()),
                         index=list(train_df["ID"].unique()))
    pos_ids=list(train_df[train_df["any"]==1].index)
    neg_ids=list(train_df[train_df["any"]==0].index)
    #random.shuffle(pos_ids)
    #random.shuffle(neg_ids)
    
    #pl,nl=len(pos_ids),len(neg_ids)
    #train_idx,valid_idx=pos_ids[:int(0.8*pl)]+neg_ids[:int(0.4*nl)],pos_ids[int(0.8*pl):]+neg_ids[int(0.4*nl):int(0.5*nl)]
    #return train_df, train_idx, valid_idx
    return train_df,pos_ids, neg_ids

train_df_dir="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv"
train_df, pos_ids, neg_ids=read_training_data(train_df_dir)

Visaulizing the total number of different types.

In [ ]:
len(train_df)

In [ ]:
height=list(train_df.sum().values)
height.append(len(neg_ids))
label=list(train_df.columns)
label.append("none")
plt.figure(figsize=(20,9))
plt.bar(x=[i*0.15 for i in range(1,8)],height=height,width=0.1,tick_label=label,color=["r","b"])
epidural_ids=list(train_df[train_df["epidural"]==1].index)
plt.figure(figsize=(18,9))
samples=random.sample(epidural_ids,10)

We define the augmentation types for every image in training set.

Helper function, GenerateAugmentationParameter, which generate randomly the information for every image in image_ids, what kind of augmentations are taken. 

In [ ]:
def GenerateAugmentationParameter(aug):
    
    p=1   
    
    if aug=="rotation":
        
        p=random.uniform(-60,+60)
        
    if aug=='scaling':
        
        p=random.uniform(1.2,1.8)
        
    if aug=='cropping':
        
        p=random.uniform(0.1,0.2)
        
    if aug=='translate':
        
        x=random.uniform(-0.2,+0.2)
        y=random.uniform(-0.2,+0.2)
        
        p={"x":x, "y":y}
        
    return p
        
    
    


def GenerateAugmentations(image_ids,df):
    
    augmentations=["windowing", "flipping", "rotation", "scaling", "cropping", "translate"]
    augs={}
    
    for i in image_ids:
    
        if df.loc[i,"epidural"]==1:
            random_aug_types=augmentations
        
        elif df.loc[i,"any"]==1: 
            random_aug_types=random.sample(augmentations[1:],3)
            random_aug_types.append("windowing")
        else:    
            #random_aug_types=random.sample(augmentations[1:],1)
            #random_aug_types.append("windowing")
            random_aug_types=["windowing"]

        augs[i]={t: GenerateAugmentationParameter(t) for t in random_aug_types}
    
    return augs
    
    
image_ids=list(train_df.index)
augs=GenerateAugmentations(image_ids,train_df)    
        

Functions for building the ResNet50 model, with ELU activation and l2 regularization.

In [ ]:
import numpy as np
import warnings
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as backend
import keras.backend as K

from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras_applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs
from keras import regularizers
from keras.layers import ELU

def identity_block(input_tensor, kernel_size, filters, stage, block, reg=None,elu_alpha=0.1):
    
    filters1, filters2, filters3=filters
    if K.image_data_format()=='channels_last':
        bn_axis=3
    else:
        bn_axis=1
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'    
    
    x=Conv2D(filters1,
             (1,1),
             name=conv_name_base+'2a',
             kernel_regularizer=reg)(input_tensor)
    x=BatchNormalization(axis=bn_axis, name=bn_name_base+'2a')(x)
    x=ELU(alpha=elu_alpha)(x)
    
    
    x=Conv2D(filters2, kernel_size, padding='same',
             name=conv_name_base+'2b',
             kernel_regularizer=reg)(x)
    x=BatchNormalization(axis=bn_axis,name=bn_name_base+'2b')(x)
    x=ELU(alpha=elu_alpha)(x)
    
    
    x=Conv2D(filters3,(1,1),name=conv_name_base+'2c',
            kernel_regularizer=reg)(x)
    x=BatchNormalization(axis=bn_axis,
                        name=bn_name_base+'2c')(x)
    
    x=layers.add([x,input_tensor])
    x=ELU(alpha=elu_alpha)(x)
    
    return x
    
    

def conv_block(input_tensor,kernel_size,filters,
               stage,block,reg=None,strides=(2,2),elu_alpha=0.1):
    
    filters1,filters2,filters3=filters
    if K.image_data_format()=='channels_last':
        bn_axis=3
        
    else:
        bn_axis=1
    
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'
    
    
    x=Conv2D(filters1,(1,1),strides=strides,
            name=conv_name_base+'2a',
             kernel_regularizer=reg)(input_tensor)
    x=BatchNormalization(axis=bn_axis,name=bn_name_base+'2a')(x)
    x=ELU(alpha=elu_alpha)(x)
    
    
    x=Conv2D(filters2, kernel_size, padding='same',
            name=conv_name_base+'2b',
            kernel_regularizer=reg)(x)
    x=BatchNormalization(axis=bn_axis,name=bn_name_base+'2b')(x)
    x=ELU(alpha=elu_alpha)(x)
    
    x=Conv2D(filters3,(1,1),name=conv_name_base+'2c',
             kernel_regularizer=reg)(x)
    x=BatchNormalization(axis=bn_axis,name=bn_name_base+'2c')(x)
    
    shortcut=Conv2D(filters3,(1,1),strides=strides,
                   name=conv_name_base+'1',
                   kernel_regularizer=reg)(input_tensor)
    shortcut=BatchNormalization(axis=bn_axis,name=bn_name_base+'1')(shortcut)
    
    x=layers.add([x,shortcut])
    x=ELU(alpha=elu_alpha)(x)
    
    return x

def ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000,
            reg=None,
            elu_alpha=0.1):
    
    WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
    WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

    
    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
            
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    
    
    x=ZeroPadding2D((3,3))(img_input)
    x=Conv2D(64,(7,7),strides=(2,2),name='conv1',
            kernel_regularizer=reg)(x)
    x=BatchNormalization(axis=bn_axis,name="bn_conv1")(x)
    x=ELU(alpha=elu_alpha)(x)
    x=MaxPooling2D((3,3),strides=(2,2))(x)
    
    x=conv_block(x,3,[64,64,256],stage=2,block='a',strides=(1,1),reg=reg)
    x=identity_block(x,3,[64,64,256],stage=2,block='b',reg=reg)
    x=identity_block(x,3,[64,64,256],stage=2,block='c',reg=reg)
    
    x=conv_block(x,3,[128,128,512],stage=3,block='a',reg=reg)
    x=identity_block(x,3,[128,128,512],stage=3,block='b',reg=reg)
    x=identity_block(x,3,[128,128,512],stage=3,block='c',reg=reg)
    x=identity_block(x,3,[128,128,512],stage=3,block='d',reg=reg)
    
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a',reg=reg)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b',reg=reg)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c',reg=reg)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d',reg=reg)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e',reg=reg)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f',reg=reg)

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a',reg=reg)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b',reg=reg)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c',reg=reg)

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path,by_name=True)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model

In [ ]:
class Augmentator:
    
    def __init__(self,IDs,augs,images_dir=train_images_dir,size=(512,512)):
        self.IDs=IDs
        self.augs=augs
        self.images_dir=train_images_dir
        self.augmentations={"windowing": self.bsb_windowing,
                            "flipping": self.left_right_flipping,
                            "rotation": self.random_rotation,
                            "scaling": self.scaling,
                            "cropping": self.cropping,
                            "translate": self.translate}
        self.augs=augs
        self.size=size
    
    def get_hu_image(self,img):
        img_array=img.pixel_array
        img_array=img.RescaleSlope*img_array+img.RescaleIntercept
        return img_array

    def windowing_with_sigmoid(self,img,center,width,ue=np.log(254)):
        #img should be already converted to hu image.
        #Rescaling w.r.t center, width
        z=ue*2*(img-center)/width
        z=np.clip(z,-20,+20)
        img=1/(1+np.exp(-z))
    
        if np.max(img)!=np.min(img):
            img=(img-np.min(img))/(np.max(img)-np.min(img))
        return img

    def bsb_windowing(self,ID,*arg):
        #Read the dcm image
        img=pydicom.dcmread(self.images_dir+ID+".dcm")
    
        #Get the hu image:
        img=self.get_hu_image(img)
    
        #Get the brain, subdural,bone windowing of the image
        brain_img=self.windowing_with_sigmoid(img, 40, 80)
        subdural_img=self.windowing_with_sigmoid(img, 80, 200)
        bone_img=self.windowing_with_sigmoid(img, 600, 2000)
    
        #Concatenate them together, each windowing is a channel
        bsb_img=np.zeros((brain_img.shape[0],brain_img.shape[1],3))
        bsb_img[:,:,0]=brain_img
        bsb_img[:,:,1]=subdural_img
        bsb_img[:,:,2]=bone_img
    
        if self.size != (512, 512):
           # resize image
           bsb_img = cv2.resize(bsb_img, self.size, interpolation=cv2.INTER_LINEAR)
        
        return bsb_img
    
    
    def left_right_flipping(self,ID,*arg):
        img=self.bsb_windowing(ID,self.size)
        aug=iaa.flip.Fliplr(1.0)
        img=aug.augment_image(img)
        return img
    
    
    def random_rotation(self,ID,rotate=None):
        #rotate: the angle, read from self.labels or just randomly generated if not labeled.
        img=self.bsb_windowing(ID,self.size)
        aug=iaa.geometric.Affine(rotate=rotate)
        img=aug.augment_image(img)
        return img
    

    def scaling(self, ID, scale=None):
        img=self.bsb_windowing(ID,self.size)
        aug=iaa.geometric.Affine(scale=scale)
        img=aug.augment_image(img)    
        return img
    
    
    def cropping(self, ID, percent=None):
        img=self.bsb_windowing(ID,self.size)
        aug=iaa.size.Crop(percent=percent)
        img=aug.augment_image(img)
        return img
    

    def translate(self, ID, translate_percent=None):
        img=self.bsb_windowing(ID,self.size)
        aug=iaa.geometric.Affine(translate_percent=translate_percent)
        img=aug.augment_image(img)
        return img
    
    
    def ImageAugmentation(self, ID,aug='windowing'):
    
        
        
        parameter=self.augs[ID][aug]
        img=self.augmentations[aug](ID, parameter )
        return img

In [ ]:

class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs,aug_IDs, labels, augmentator, batch_size=1, img_size=(512, 512), 
                 img_dir=train_images_dir,*args, **kwargs):
        self.list_IDs = list_IDs
        self.aug_IDs=aug_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()
        self.augmentator=augmentator
    
    
    def __len__(self):
        return int(ceil(len(self.aug_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        aug_IDs_temp = [self.aug_IDs[k] for k in indices]
        X, Y = self.__data_generation(aug_IDs_temp)
        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.aug_IDs))
        np.random.shuffle(self.indices)

    def __data_generation(self, aug_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 3))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, aug_ids in enumerate(aug_IDs_temp):
            X[i,] = self.augmentator.ImageAugmentation(aug_ids[0],aug=aug_ids[1])
            Y[i,] = self.labels.loc[aug_ids[0]].values
        
        return X, Y
    
    
    
    
    
    
    
class TestDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, size=(512, 512), 
                 images_dir=test_images_dir, *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.size =size
        self.images_dir = images_dir
        self.on_epoch_end()
        
   
    
    def get_hu_image(self,img):
        img_array=img.pixel_array
        img_array=img.RescaleSlope*img_array+img.RescaleIntercept
        return img_array

    def windowing_with_sigmoid(self,img,center,width,ue=np.log(254)):
        #img should be already converted to hu image.
        #Rescaling w.r.t center, width
        z=ue*2*(img-center)/width
        z=np.clip(z,-20,+20)
        img=1/(1+np.exp(-z))
    
        if np.max(img)!=np.min(img):
            img=(img-np.min(img))/(np.max(img)-np.min(img))
        return img

    def bsb_windowing(self,ID,*arg):
        #Read the dcm image
        img=pydicom.dcmread(self.images_dir+ID+".dcm")
    
        #Get the hu image:
        img=self.get_hu_image(img)
    
        #Get the brain, subdural,bone windowing of the image
        brain_img=self.windowing_with_sigmoid(img, 40, 80)
        subdural_img=self.windowing_with_sigmoid(img, 80, 200)
        bone_img=self.windowing_with_sigmoid(img, 600, 2000)
    
        #Concatenate them together, each windowing is a channel
        bsb_img=np.zeros((brain_img.shape[0],brain_img.shape[1],3))
        bsb_img[:,:,0]=brain_img
        bsb_img[:,:,1]=subdural_img
        bsb_img[:,:,2]=bone_img
    
        if self.size != (512, 512):
           # resize image
           bsb_img = cv2.resize(bsb_img, self.size, interpolation=cv2.INTER_LINEAR)
        
        return bsb_img
        

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X = self.__data_generation(list_IDs_temp)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.size, 3))
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = self.bsb_windowing(ID)
        
        return X

In [ ]:
def _weighted_log_loss(y_true, y_pred):

    
    y_pred = keras.backend.clip(y_pred, keras.backend.epsilon(), 1.0-keras.backend.epsilon())
    
    weights=np.array([1.0,1.0,1.0,1.0,1.0,2.0])/7.0
    out = -(         y_true  * keras.backend.log(      y_pred) 
            +(1.0 - y_true) * keras.backend.log(1.0 - y_pred) )
    
    return keras.backend.sum(out*weights, axis=-1)


class MyDeepModel:
    
    def __init__(self, input_dims, batch_size=5, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", l2_reg=0,verbose=1,elu_alpha=0.1):
        
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self.l2_reg=l2_reg
        self.elu_alpha=elu_alpha
        self._build()

    def _build(self):
        
        reg=keras.regularizers.l2(self.l2_reg)
        
        #We build a ResNet50 model with L2 regularization
        ResNetModel=ResNet50(input_tensor=None,include_top=False, weights=self.weights,
                               input_shape=(*self.input_dims, 3),
                             reg=reg,elu_alpha=self.elu_alpha)
        
        x=ResNetModel.output
        
        x = keras.layers.GlobalAveragePooling2D(name='avg_pool_final')(x)

        x = keras.layers.Dense(6, activation="sigmoid", name='dense_output')(x)

        self.model = keras.models.Model(inputs=ResNetModel.input, outputs=x)

        self.model.compile(loss=_weighted_log_loss, optimizer=keras.optimizers.Adam(0.0))
        
    """TrainDataGenerator(
                train_idx,
                aug_ids, 
                df.loc[train_idx], 
                augmentator,
                self.batch_size, 
                self.input_dims,
                img_dir=train_images_dir,
            ),"""
    
    def fit(self, df, train_idx, aug_ids, augmentator,global_epoch): 
        self.model.fit_generator(
            TrainDataGenerator(
                train_idx,
                aug_ids, 
                df.loc[train_idx], 
                augmentator,
                self.batch_size, 
                self.input_dims,
                img_dir=train_images_dir,
            ),
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4,
            callbacks=[
                keras.callbacks.LearningRateScheduler(
                    lambda global_epoch: self.learning_rate * pow(self.decay_rate, floor(global_epoch / self.decay_steps)))
            ]
        )
    
    def predict(self, df, test_idx, img_dir):
        predictions = \
          self.model.predict_generator(
            TestDataGenerator(
                test_idx, 
                None, 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4
        )
        
        return predictions[:df.loc[test_idx].shape[0]]
        
    
    def save(self, path):
        self.model.save_weights(path)
    
    def load(self, path,by_name=False):
        self.model.load_weights(path,by_name=by_name)


In [ ]:
def read_testset(filename="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv"):
    df=import_data(filename)
    df["SubType"]=df["ID"].apply(lambda x: x.rsplit("_",1)[-1])
    df["ID"]=df["ID"].apply(lambda x: x.rsplit("_",1)[0])
    Label=np.reshape(df["Label"].values,(-1,6))
    df=pd.DataFrame(Label,
                    columns=list(df["SubType"].unique()),
                    index=list(df["ID"].unique()))
    
    
    return df
    
test_df = read_testset()


In [ ]:
len(test_df)

In [ ]:

"""
TrainDataGenerator(
                train_idx,
                aug_ids, 
                df.loc[train_idx], 
                augmentator,
                self.batch_size, 
                self.input_dims,
                img_dir,
            )
list_IDs, aug_IDs, labels, augmentator, batch_size=1, img_size=(512, 512)            
"""
"""
augmentator=Augmentator(list(train_df.index),augs,size=(256,256))
random.shuffle(pos_ids)
random.shuffle(neg_ids)
train_df=train_df.loc[pos_ids+neg_ids]
train_df.to_csv("train_df.csv")
aug_df=pd.DataFrame(list(augs.values()),index=list(augs.keys()))
aug_df.to_csv("augmentation_parameters.csv")
"""



In [ ]:
"""
def weighted_log_loss_metric(trues, preds): # add 0.2
    class_weights = [1.0/7.0, 1.0/7.0, 1.0/7.0, 1.0/7.0, 1.0/7.0, 2.0/7.0]
    # higher epsilon than competition metric
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss_subtypes = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_weighted = np.sum(loss_subtypes*class_weights,axis=1)

    return -loss_weighted.mean()

#def run(model, df, pos_ids, neg_ids, epochs):
    
    #valid_predictions = []
    #test_predictions = []
    #valid_losses=[]

model = MyDeepModel(input_dims=(256, 256), batch_size=16, learning_rate=1.5e-5, 
                    decay_rate=0.7, decay_steps=1, weights="imagenet", l2_reg=0, verbose=2, elu_alpha=0.1)
                        
valid_idx=pos_ids[:len(pos_ids)//10]+neg_ids[:len(neg_ids)//10]
train_pos_idx,train_neg_idx=pos_ids[len(pos_ids)//10:2*(len(pos_ids)//10)],neg_ids[len(neg_ids)//10:2*(len(neg_ids)//10)]
train_idx=train_pos_idx+train_neg_idx
train_aug_idx=[(idx,j) for idx in train_idx for j in augs[idx]]

for global_epoch in range(10):
                        
    model.fit(train_df,train_idx,train_aug_idx,augmentator,global_epoch)
    model.save("ResNet50"+"_"+str(global_epoch)+".h5")
    
    if global_epoch>5:
        prediction=model.predict(train_df, valid_idx, _TRAIN_IMAGES)    
        #valid_predictions.append(model.predict(df, valid_idx, _TRAIN_IMAGES))
        valid_loss=weighted_log_loss_metric(train_df.loc[valid_idx].values, 
                                         prediction)
        #valid_losses.append(valid_loss)
        print("validation loss:"+str(valid_loss))    
"""

            

In [ ]:
"""
train_df=import_data("../input/kernel3c163cba40/train_df.csv")
train_df.rename({"Unnamed: 0": "ID"},axis="columns",inplace=True)
train_df.set_index("ID",inplace=True)
pos_ids=list(train_df[train_df["any"]==1].index)
neg_ids=list(train_df[train_df["any"]==0].index)

aug_df=import_data("../input/kernel3c163cba40/augmentation_parameters.csv")
aug_df.rename({"Unnamed: 0": "ID"},axis="columns",inplace=True)
aug_df.set_index("ID",inplace=True)

augs_notnull=aug_df.notnull()
augs={ID: {c: aug_df.loc[ID,c] for c in ['cropping','windowing','rotation','scaling','flipping'] if augs_notnull.loc[ID,c]==True} for ID in aug_df.index}

translate=aug_df.translate[aug_df.translate.notnull()]
for i in translate.index:
    x=float(translate[i].split()[1][:-1])
    y=float(translate[i].split()[3][:-1])
    augs[i]['translate']={'x':x,'y':y}
    
del augs_notnull, aug_df

def weighted_log_loss_metric(trues, preds): # add 0.2
    class_weights = [1.0/7.0, 1.0/7.0, 1.0/7.0, 1.0/7.0, 1.0/7.0, 2.0/7.0]
    # higher epsilon than competition metric
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss_subtypes = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_weighted = np.sum(loss_subtypes*class_weights,axis=1)

    return -loss_weighted.mean()

#def run(model, df, pos_ids, neg_ids, epochs):
    
    #valid_predictions = []
    #test_predictions = []
    #valid_losses=[]

model = MyDeepModel(input_dims=(256, 256), batch_size=16, learning_rate=1.5e-5, 
                    decay_rate=1, decay_steps=1, weights=None, l2_reg=0, verbose=2, elu_alpha=0.1)

model.load("../input/kernel3c163cba40/ResNet50_4.h5",by_name=True)
           
valid_idx=pos_ids[:len(pos_ids)//10]+neg_ids[:len(pos_ids)//10]
train_idx_pos,train_idx_neg=pos_ids[len(pos_ids)//10:],neg_ids[len(neg_ids)//10:]
augmentator=Augmentator(list(train_df.index),augs,size=(256,256))

for global_epoch in range(2,8):
        
    subsample_pos_idx=train_idx_pos[(len(train_idx_pos)//8)*global_epoch:(len(train_idx_pos)//8)*(global_epoch+1)]
    subsample_neg_idx=train_idx_neg[(len(train_idx_neg)//16)*global_epoch:(len(train_idx_neg)//16)*(global_epoch+1)]
                
    subsample_train_idx=subsample_pos_idx+subsample_neg_idx
    
    
    subsample_aug_idx=[(idx,j) for idx in subsample_train_idx for j in augs[idx]]
    
    model.fit(train_df,subsample_train_idx,subsample_aug_idx,augmentator,1)
    
    if global_epoch%2==1:
        
        model.save("ResNet50"+"_"+str(global_epoch)+".h5")
        
prediction=model.predict(train_df, valid_idx, _TRAIN_IMAGES)    
        #valid_predictions.append(model.predict(df, valid_idx, _TRAIN_IMAGES))
valid_loss=weighted_log_loss_metric(train_df.loc[valid_idx].values, 
                                         prediction)
        #valid_losses.append(valid_loss)
        
print(valid_loss)

"""




In [ ]:
model = MyDeepModel(input_dims=(256, 256), batch_size=16, learning_rate=1.5e-5, 
                    decay_rate=1, decay_steps=1, weights=None, l2_reg=0, verbose=2, elu_alpha=0.1)
model.load("../input/kernel3c163cba40/ResNet50_4.h5",by_name=True)



In [ ]:
prediction=model.predict(test_df, list(test_df.index), test_images_dir)    

In [ ]:
test_df.iloc[:, :] = prediction
test_df.to_csv("predition.csv",index=False)
test_df = test_df.stack().reset_index()
test_df["ID"]=test_df["level_0"].astype(str)+"_"+test_df["level_1"].astype(str)
test_df.drop(["level_0","level_1"],axis=1,inplace=True)
test_df.columns=['ID','Label']
test_df.rename({0:"Label"},axis=1)
test_df.to_csv("submission.csv",index=False)

In [ ]:
df=import_data("../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv")

In [ ]:
df2=import